In [8]:
import pandas as pd
from passlib.context import CryptContext
from sources.utils import get_root
import os
from pathlib import Path

In [5]:

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
users = {

    "daniel" :{
        "username": "daniel",
        "password": pwd_context.hash('datascientest'),
    },
    "john" : {
        "username" :  "john",
        "password" : pwd_context.hash('secret'),
    },
    "lucie" :{
        "username": "lucie",
        "password" : pwd_context.hash('ravie')
    }
}

In [6]:
# Open a file in write mode.
with open('users.csv', 'w') as f:
    # Write all the dictionary keys in a file with commas separated.
    f.write(','.join(users[0].keys()))
    f.write('\n') # Add a new line
    for row in users:
        # Write the values in a row.
        f.write(','.join(str(x) for x in row.values()))
        f.write('\n') # Add a new line

KeyError: 0

In [13]:
df= pd.read_csv(os.path.join(get_root(), 'users.csv'))

In [14]:
df.columns

Index(['username', 'password'], dtype='object')

In [12]:
csv_path = os.path.join(get_root(), 'users.csv')
print(csv_path)
#generate_config(csv_path, "config.py")

C:\Users\lucie\SmashTheOdds\users.csv


In [15]:
from sources.data_pipeline import global_transformer
from sources.preprocessor import get_match_info
from sources.utils import get_last_model, get_response
import joblib


def get_pred(match_id: int):
    model = joblib.load(get_last_model())
    match_info = get_match_info(match_id=match_id)
    try:
        data = global_transformer(match_info[['player1_id', 'player2_id']])
        return get_response(model=model, data=data)
    except TypeError:
        return match_info

get_pred(42024053)

https://api.sportradar.com/tennis/trial/v2/fr/matches/sr:match:42024053/summary.json
200


C:\Users\lucie\SmashTheOdds\sources\utils.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[['player1_id', 'player2_id']] = dataframe[['player1_id', 'player2_id']].replace(player_rank_mapping)
C:\Users\lucie\SmashTheOdds\sources\utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['player1_id'] = dataframe['player1_id'].replace(to_replace='^.*$', value=unranked, regex=True)
C:\Users\lucie\SmashTheOdds\sources\utils.py:60: SettingWithCopyWarning: 
A value is trying to be set on a

{'classe': 1, 'proba': [0.5, 0.5], 'odds': [2.02, 1.98]}

In [18]:
# -*- coding: utf-8 -*-
"""
Created on 24/05/2023 22:07
@author: GiovanniMINGHELLI
"""
import os
from typing import List

import pandas as pd
from tqdm import tqdm
from datetime import date
from sources.get_data import Tennis
from sources.utils import get_number_in_id, check_file_modification, get_root
from sources.week_calendar import get_next_seven_days

today = date.today()


def prep_daily_results(year=today.year, month=today.month, day=today.day):
    return Tennis().get_daily_results(year=year, month=month, day=day)


def get_weekly_schedule():
    df_list = []
    for date in get_next_seven_days():
        year, month, day = map(int, date.split('-'))
        response = Tennis().get_daily_schedule(year=year, month=month, day=day)
        if response.status_code != 200:
            return response.json()
        response_data = response.json()
        if 'sport_events' in response_data and response_data['sport_events']:
            df_response = pd.json_normalize(response_data['sport_events'])
            df_response = pd.json_normalize(response.json()['sport_events'])
            df_response[['player1', 'player2']] = df_response['competitors'].apply(pd.Series)
            player1_df = pd.json_normalize(df_response['player1']).add_prefix('player1_') # type: ignore
            player2_df = pd.json_normalize(df_response['player2']).add_prefix('player2_') # type: ignore

            df = pd.concat([df_response, player1_df, player2_df], axis=1)
            df_list.append(df)
    return pd.concat(df_list, axis=0, ignore_index=True)




In [19]:
get_weekly_schedule(

https://api.sportradar.com/tennis/trial/v2/fr/schedules/2023-08-10/schedule.json
200
https://api.sportradar.com/tennis/trial/v2/fr/schedules/2023-08-11/schedule.json
200
https://api.sportradar.com/tennis/trial/v2/fr/schedules/2023-08-12/schedule.json
200
https://api.sportradar.com/tennis/trial/v2/fr/schedules/2023-08-13/schedule.json
200
https://api.sportradar.com/tennis/trial/v2/fr/schedules/2023-08-14/schedule.json
200
https://api.sportradar.com/tennis/trial/v2/fr/schedules/2023-08-15/schedule.json
200
https://api.sportradar.com/tennis/trial/v2/fr/schedules/2023-08-16/schedule.json
200


C:\Users\lucie\anaconda3\lib\site-packages\IPython\lib\pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
C:\Users\lucie\anaconda3\lib\site-packages\IPython\core\formatters.py:343: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,id,scheduled,start_time_tbd,status,competitors,sport_event_type,estimated,tournament_round.type,tournament_round.name,tournament_round.cup_round_match_number,...,player2_name,player2_nationality,player2_abbreviation,player2_seed,player2_bracket_number,player2_qualifier,player2_country_code,player2_players,player1_virtual,player2_virtual
0,sr:match:42697623,2023-08-10T00:10:00+00:00,False,closed,"[{'id': 'sr:competitor:19704', 'name': 'Martic...",singles,True,cup,round_of_32,1.0,...,"Sabalenka, Aryna",Neutral,SAB,2.0,64.0,away,NaN,NaN,NaN,NaN
1,sr:match:42696687,2023-08-10T00:15:00+00:00,False,closed,"[{'id': 'sr:competitor:203681', 'name': 'Korda...",singles,True,cup,round_of_32,1.0,...,"Vukic, Aleksandar",Australie,VUK,NaN,23.0,away,AUS,NaN,NaN,NaN
2,sr:match:42750361,2023-08-10T00:20:00+00:00,False,closed,"[{'id': 'sr:competitor:1042205', 'name': 'Belt...",doubles,True,cup,round_of_16,1.0,...,"Carou, I / Monzon, I",NaN,C/M,NaN,8.0,away,NaN,"[{'id': 'sr:competitor:107483', 'name': 'Monzo...",NaN,NaN
3,sr:match:42803175,2023-08-10T00:25:00+00:00,False,closed,"[{'id': 'sr:competitor:1024889', 'name': 'Kala...",doubles,True,cup,round_of_16,NaN,...,"Bondarenko, K / Yamalapalli, S",NaN,B/Y,NaN,2.0,away,NaN,"[{'id': 'sr:competitor:18742', 'name': 'Bondar...",NaN,NaN
4,sr:match:42747269,2023-08-10T00:30:00+00:00,False,closed,"[{'id': 'sr:competitor:1042089', 'name': 'Ried...",doubles,True,cup,round_of_16,1.0,...,"Hong, S / Shimabukuro, S",NaN,H/S,NaN,8.0,away,NaN,"[{'id': 'sr:competitor:97977', 'name': 'Hong, ...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,sr:match:42854099,2023-08-16T17:00:00+00:00,True,not_started,"[{'id': 'sr:competitor:837880', 'name': 'R16P9...",doubles,True,cup,round_of_16,1.0,...,R16P10,NaN,R16,NaN,NaN,away,NaN,NaN,True,True
607,sr:match:42854101,2023-08-16T17:00:00+00:00,True,not_started,"[{'id': 'sr:competitor:837868', 'name': 'R16P3...",doubles,True,cup,round_of_16,1.0,...,R16P4,NaN,R16,NaN,NaN,away,NaN,NaN,True,True
608,sr:match:42854103,2023-08-16T17:00:00+00:00,True,not_started,"[{'id': 'sr:competitor:837872', 'name': 'R16P5...",doubles,True,cup,round_of_16,1.0,...,R16P6,NaN,R16,NaN,NaN,away,NaN,NaN,True,True
609,sr:match:42854105,2023-08-16T17:00:00+00:00,True,not_started,"[{'id': 'sr:competitor:837864', 'name': 'R16P1...",doubles,True,cup,round_of_16,1.0,...,R16P2,NaN,R16,NaN,NaN,away,NaN,NaN,True,True


In [7]:
display(df_response)

NameError: name 'df_response' is not defined

In [8]:
display(df_list)

NameError: name 'df_list' is not defined

In [1]:
import sklearn

print(sklearn.__version__)


1.1.3
